In [1]:
import requests
import json
from Resources.API_Key import api_key
import pandas as pd

In [2]:
# Using the API key to access BLS data   
url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"

series_id_lp = "PRS85006092"

payload = {
    "seriesid": [series_id_lp],
    "startyear": "2018",
    "endyear": "2024",
    "registrationkey": api_key
}

response = requests.post(url, json=payload)

if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=4))
else:
    print(f"Error {response.status_code}")
    

{
    "status": "REQUEST_SUCCEEDED",
    "responseTime": 148,
    "message": [],
    "Results": {
        "series": [
            {
                "seriesID": "PRS85006092",
                "data": [
                    {
                        "year": "2024",
                        "period": "Q02",
                        "periodName": "2nd Quarter",
                        "latest": "true",
                        "value": "2.5",
                        "footnotes": [
                            {
                                "code": "R",
                                "text": "revised"
                            }
                        ]
                    },
                    {
                        "year": "2024",
                        "period": "Q01",
                        "periodName": "1st Quarter",
                        "value": "0.4",
                        "footnotes": [
                            {}
                        ]
                    },
   

In [3]:
# Converting the data into a dataframe
series_data = data['Results']['series'][0]['data']
labor_productivity_df = pd.DataFrame(series_data)

# Formatting the data
quarter_mapping = {
    "Q01": "Q1: Jan 1 - Mar 31",
    "Q02": "Q2: Apr 1 - Jun 30",
    "Q03": "Q3: Jul 1 - Sep 30",
    "Q04": "Q4: Oct 1 - Dec 31"
}

# Extra formatting for the data that came from the API request
labor_productivity_df['quarter'] = labor_productivity_df['period'].map(quarter_mapping)
labor_productivity_df['year'] = pd.to_numeric(labor_productivity_df['year'])
labor_productivity_df['percent_change'] = pd.to_numeric(labor_productivity_df['value'])
labor_productivity_df = labor_productivity_df[['year', 'quarter', 'percent_change']]
labor_productivity_df = labor_productivity_df.sort_values(by=['year', 'quarter']).reset_index(drop=True)

In [4]:
# Reading in the Excel file that was used for this data frame
file_path = "Resources/unit_labor_costs.xlsx"
sheet_name = "BLS Data Series"
unit_labor_costs_df = pd.read_excel(file_path)
unit_labor_costs_df = unit_labor_costs_df.melt(id_vars =["Year"],
                                               var_name="quarter",
                                               value_name="percent_change")

#Formatting the data so that it matches the previous data frame
quarter_mapping2 = {
    "Qtr1": "Q1: Jan 1 - Mar 31",
    "Qtr2": "Q2: Apr 1 - Jun 30",
    "Qtr3": "Q3: Jul 1 - Sep 30",
    "Qtr4": "Q4: Oct 1 - Dec 31"
}
unit_labor_costs_df["quarter"] = unit_labor_costs_df["quarter"].map(quarter_mapping2)
unit_labor_costs_df = unit_labor_costs_df.sort_values(by=["Year", "quarter"]).reset_index(drop=True)
unit_labor_costs_df = unit_labor_costs_df.dropna(subset=["percent_change"])

In [5]:
# Rinse and repeat for the next excel sheet
file_path2 = "Resources/real_hourly_compensation.xlsx"
sheet_name2 = sheet_name
real_hourly_compensation_df = pd.read_excel(file_path2)
real_hourly_compensation_df = real_hourly_compensation_df.melt(id_vars = ["Year"],
                                                               var_name="quarter",
                                                               value_name="percent_change")

real_hourly_compensation_df["quarter"] = real_hourly_compensation_df["quarter"].map(quarter_mapping2)
real_hourly_compensation_df = real_hourly_compensation_df.sort_values(by=["Year", "quarter"]).reset_index(drop=True)
real_hourly_compensation_df = real_hourly_compensation_df.dropna(subset=["percent_change"])

In [6]:
file_path3 = "Resources/total_factor_productivity.xlsx"
sheet_name3 = sheet_name
total_factor_productivity_df = pd.read_excel(file_path3)

In [7]:
# Labor Productivity DataFrame
labor_productivity_df

,year,quarter,percent_change
0,2018,Q1: Jan 1 - Mar 31,1.5
1,2018,Q2: Apr 1 - Jun 30,0.0
2,2018,Q3: Jul 1 - Sep 30,1.2
3,2018,Q4: Oct 1 - Dec 31,-0.6
4,2019,Q1: Jan 1 - Mar 31,3.1
5,2019,Q2: Apr 1 - Jun 30,2.6
6,2019,Q3: Jul 1 - Sep 30,4.4
7,2019,Q4: Oct 1 - Dec 31,3.6
8,2020,Q1: Jan 1 - Mar 31,-0.9
9,2020,Q2: Apr 1 - Jun 30,19.9


In [8]:
# Unit Labor Costs DataFrame
unit_labor_costs_df

,Year,quarter,percent_change
0,2018,Q1: Jan 1 - Mar 31,1.1
1,2018,Q2: Apr 1 - Jun 30,2.0
2,2018,Q3: Jul 1 - Sep 30,2.8
3,2018,Q4: Oct 1 - Dec 31,2.1
4,2019,Q1: Jan 1 - Mar 31,6.5
5,2019,Q2: Apr 1 - Jun 30,-2.3
6,2019,Q3: Jul 1 - Sep 30,-4.0
7,2019,Q4: Oct 1 - Dec 31,3.4
8,2020,Q1: Jan 1 - Mar 31,11.8
9,2020,Q2: Apr 1 - Jun 30,3.7


In [9]:
# Real Hourly Compensation Data Frame
real_hourly_compensation_df

,Year,quarter,percent_change
0,2018,Q1: Jan 1 - Mar 31,-0.8
1,2018,Q2: Apr 1 - Jun 30,-0.1
2,2018,Q3: Jul 1 - Sep 30,2.2
3,2018,Q4: Oct 1 - Dec 31,-0.2
4,2019,Q1: Jan 1 - Mar 31,8.8
5,2019,Q2: Apr 1 - Jun 30,-2.7
6,2019,Q3: Jul 1 - Sep 30,-1.1
7,2019,Q4: Oct 1 - Dec 31,4.1
8,2020,Q1: Jan 1 - Mar 31,9.2
9,2020,Q2: Apr 1 - Jun 30,28.8


In [10]:
# Total Factor Productivity Data Frame
total_factor_productivity_df

,Year,Annual
0,2018,100.633
1,2019,101.987
2,2020,101.756
3,2021,105.053
4,2022,103.343
5,2023,104.107
